In [1]:
pwd

'/Users/peterhaglich/Dropbox/Work/IARPA/Mercury/peterhaglich/mercury-challenge/src/ExpressScore/notebooks'

In [3]:
%matplotlib inline

In [4]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from dateutil.parser import parse
import datetime
import calendar
import json
import os
import re

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use("fivethirtyeight")

from collections import Counter

from geopy.distance import distance

In [6]:
EVENT_TYPE = "Military Action"
EVT_ABBR = "MA"

month_str = "May 2018"
month_path_str = "_".join(month_str.split(" "))
MERC_CHALLENGE_HOME = os.path.abspath("../../../..")

MERC_HOME = os.path.join(MERC_CHALLENGE_HOME, "..", "mercury")
EXPRESS_SCORE_PATH = os.path.join(MERC_CHALLENGE_HOME, "src", "ExpressScore")
print(os.listdir(EXPRESS_SCORE_PATH))
ES_TEST_RESOURCE_PATH = os.path.join(EXPRESS_SCORE_PATH, "resources", "test", "lb_ma_may_2018")
DATA_HOME = os.path.join(MERC_HOME, "data")
WARN_PATH = os.path.join(DATA_HOME, "baserate_warnings", "MANSA")
month_warn_path = os.path.join(WARN_PATH, month_path_str)
MANSA_GSR_PATH = os.path.join(MERC_CHALLENGE_HOME, "data", "gsr", "ma_gsr")

['.DS_Store', '__init__.py', '__pycache__', 'main', 'notebooks', 'README.md', 'resources', 'test']


In [6]:
br_warn_filename = "Baserate_MANSA_{}.json".format(month_path_str)
br_warn_path = os.path.join(WARN_PATH, br_warn_filename)
with open(br_warn_path, "r", encoding="utf8") as f:
    br_warn = json.load(f)
br_warn = [w for w in br_warn["payload"] if w["Event_Type"] == "Military Action"]
br_country_counts = Counter([w["Country"] for w in br_warn])
print(br_country_counts)

Counter({'Syria': 1685, 'Iraq': 529, 'Lebanon': 42, 'Egypt': 35, 'Yemen': 5, 'Saudi Arabia': 5})


In [7]:
gsr_filename = "MA_{}.json".format(month_path_str)
gsr_path = os.path.join(MANSA_GSR_PATH, gsr_filename)
with open(gsr_path, "r", encoding="utf8") as f:
    gsr = json.load(f)
ma_gsr = [e for e in gsr if e["Event_Type"] == "Military Action"]
gsr_country_counts = Counter([e["Country"] for e in ma_gsr])
print(gsr_country_counts)

Counter({'Syria': 884, 'Iraq': 439, 'Saudi Arabia': 13, 'Egypt': 10, 'Yemen': 6, 'Lebanon': 4})


In [8]:
test_evt = [e for e in ma_gsr if e["Country"] == "Saudi Arabia"][0]
test_evt

{'Actor': 'Royal Saudi Military',
 'Approximate_Location': 'False',
 'City': 'Najrān',
 'Country': 'Saudi Arabia',
 'Earliest_Reported_Date': '2018-05-06',
 'Event_Date': '2018-05-06',
 'Event_ID': 'MN267278',
 'Event_Subtype': 'Conflict',
 'Event_Type': 'Military Action',
 'First_Reported_Link': 'https://www.elnashra.com/news/show/1207260/%D8%A7%D9%84%D8%AF%D9%81%D8%A7%D8%B9-%D8%A7%D9%84%D8%AC%D9%88%D9%8A-%D8%A7%D9%84%D8%B3%D8%B9%D9%88%D8%AF%D9%8A-%D9%8A%D8%AF%D9%85%D8%B1-%D8%B5%D8%A7%D8%B1%D9%88%D8%AE%D9%8A%D9%86-%D8%A8%D8%A7%D9%84%D9%8A%D8%B3%D8%AA%D9%8A%D9%8A%D9%86-%D8%A3%D8%B7%D9%84%D9%82%D9%87%D9%85',
 'GSS_Link': 'https://www.elnashra.com/news/show/1207260/%D8%A7%D9%84%D8%AF%D9%81%D8%A7%D8%B9-%D8%A7%D9%84%D8%AC%D9%88%D9%8A-%D8%A7%D9%84%D8%B3%D8%B9%D9%88%D8%AF%D9%8A-%D9%8A%D8%AF%D9%85%D8%B1-%D8%B5%D8%A7%D8%B1%D9%88%D8%AE%D9%8A%D9%86-%D8%A8%D8%A7%D9%84%D9%8A%D8%B3%D8%AA%D9%8A%D9%8A%D9%86-%D8%A3%D8%B7%D9%84%D9%82%D9%87%D9%85',
 'Latitude': 17.5,
 'Longitude': 44.1667,
 'News_Source

In [9]:
lb_gsr = [e for e in ma_gsr if e["Country"] == "Lebanon"]
print(len(lb_gsr))
for i, e in enumerate(lb_gsr):
    new_id = "MN{}".format(i)
    e["Event_ID"] = new_id
out_filename = "test_lb_gsr.json"
out_path = os.path.join(ES_TEST_RESOURCE_PATH, out_filename)
with open(out_path, "w") as f:
    json.dump(lb_gsr, f, ensure_ascii=False, indent=2)

4


In [10]:
lb_warn = [w for w in br_warn if w["Country"] == "Lebanon"]
first_date = parse("2018-05-12")
last_date = parse("2018-05-24")
lb_warn = [w for w in lb_warn if parse(w["Event_Date"]) <= last_date
           and parse(w["Event_Date"]) >= first_date]
for w in lb_warn:
    if w["Event_Subtype"] == "Armed Conflict":
        w["Event_Subtype"] = "Conflict"
    keys_ = list(w.keys())
    for k in keys_:
        if re.findall("Target", k):
            del w[k]
print(len(lb_warn))
for i, w in enumerate(lb_warn):
    new_id = "BR_{}".format(i)
    w["Warning_ID"] = new_id
out_filename = "test_lb_warnings.json"
out_path = os.path.join(ES_TEST_RESOURCE_PATH, out_filename)
with open(out_path, "w") as f:
    json.dump(lb_warn, f, ensure_ascii=False, indent=2)

17


In [11]:
lb_gsr[0]

{'Actor': 'Lebanese Military',
 'Approximate_Location': 'False',
 'City': 'El Hermel',
 'Country': 'Lebanon',
 'Earliest_Reported_Date': '2018-05-14',
 'Event_Date': '2018-05-14',
 'Event_ID': 'MN0',
 'Event_Subtype': 'Force Posture',
 'Event_Type': 'Military Action',
 'First_Reported_Link': 'http://nna-leb.gov.lb/en/show-news/91098/Two-Syrian-siblings-arrested-in-Hermel-for-belonging-to-Daesh',
 'GSS_Link': 'http://nna-leb.gov.lb/en/show-news/91098/Two-Syrian-siblings-arrested-in-Hermel-for-belonging-to-Daesh',
 'Latitude': 34.3942,
 'Longitude': 36.3847,
 'News_Source': 'NNA',
 'Other_Links': 'https://www.elnashra.com/news/show/1209404/%D8%AA%D9%88%D9%82%D9%8A%D9%81-%D8%B3%D9%88%D8%B1%D9%8A%D9%8A%D9%86-%D8%A7%D9%84%D9%87%D8%B1%D9%85%D9%84-%D9%84%D8%A7%D9%86%D8%AA%D9%85%D8%A7%D8%A6%D9%87%D9%85%D8%A7-%D8%A5%D9%84%D9%89-%D8%AA%D9%86%D8%B8%D9%8A%D9%85-%D8%AF%D8%A7%D8%B9%D8%B4;',
 'Revision_Date': '2018-06-07',
 'State': 'Baalbek-Hermel'}

In [12]:
lb_warn[0]

{'Actor': 'Hezbollah',
 'City': 'Aarsâl',
 'Country': 'Lebanon',
 'Event_Date': '2018-05-15',
 'Event_Subtype': 'Conflict',
 'Event_Type': 'Military Action',
 'Latitude': 34.1794,
 'Longitude': 36.4208,
 'Probability': 0.8143660227,
 'State': 'Baalbek-Hermel',
 'Warning_ID': 'BR_0',
 'timestamp': '2018-05-07T16:40:34.0'}

In [13]:
def dist(warn, evt):
    w_lat = warn["Latitude"]
    w_long = warn["Longitude"]
    e_lat = evt["Latitude"]
    e_long = evt["Longitude"]
    return distance((w_lat, w_long), (e_lat, e_long)).km
def dist_to_warn(w, gsr_list):
    return [dist(w, e) for e in gsr_list]
def date_diff(warn, evt):
    w_date = parse(warn["Event_Date"])
    e_date = parse(evt["Event_Date"])
    delta = (w_date - e_date).days
    return delta
def date_diff_to_warn(w, gsr_list):
    return [date_diff(w, e) for e in gsr_list]
def es_match(warn, evt):
    w_es = warn["Event_Subtype"]
    e_es = evt["Event_Subtype"]
    return (w_es == e_es)
def es_match_to_warn(w, gsr_list):
    return [int(es_match(w, e)) for e in gsr_list]
def actor_match(warn, evt):
    e_actors = evt["Actor"].split(";")
    return (warn["Actor"] in e_actors)
def actor_match_to_warn(w, gsr_list):
    return [int(actor_match(w, e)) for e in gsr_list]
def ls(dist):
    return 1 - dist/100.0
ls_vfunc = np.vectorize(ls)
def ds(date_diff):
    return 1 - abs(date_diff)/4.0
ds_vfunc = np.vectorize(ds)

In [14]:
print(date_diff(lb_warn[0], lb_gsr[0]))
print(date_diff_to_warn(lb_warn[0], lb_gsr))

print(dist(lb_warn[0], lb_gsr[0]))

print(dist_to_warn(lb_warn[0], lb_gsr))
print(es_match(lb_warn[0], lb_gsr[0]))

print(es_match_to_warn(lb_warn[0], lb_gsr))
print(actor_match(lb_warn[0], lb_gsr[0]))
print(actor_match_to_warn(lb_warn[0], lb_gsr))

1
[1, -7, -7, -3]
24.057959656931487
[24.057959656931487, 59.81798090289994, 59.81798090289994, 38.80202830909005]
False
[0, 1, 0, 1]
False
[0, 0, 0, 0]


In [15]:
print(lb_warn[0]["Event_Subtype"], lb_gsr[0]["Event_Subtype"], lb_gsr[1]["Event_Subtype"])

Conflict Force Posture Conflict


In [16]:
dist_array = [dist_to_warn(w, lb_gsr) for w in lb_warn]
dist_array = np.array(dist_array).reshape(len(lb_warn), len(lb_gsr))

dist_df = pd.DataFrame(dist_array,
                       index = [w["Warning_ID"] for w in lb_warn],
                       columns = [e["Event_ID"] for e in lb_gsr])

dist_df

,MN0,MN1,MN2,MN3
BR_0,24.057960,59.817981,59.817981,38.802028
BR_1,49.409320,0.000000,0.000000,63.501466
BR_2,40.565284,13.646094,13.646094,50.546921
BR_3,44.999888,77.054362,77.054362,34.367458
BR_4,128.075720,116.237801,116.237801,71.571180
BR_5,15.340757,55.986712,55.986712,45.676614
BR_6,22.285861,61.808940,61.808940,43.142217
BR_7,24.057960,59.817981,59.817981,38.802028
BR_8,17.905095,63.568006,63.568006,52.002395
BR_9,168.434708,148.882978,148.882978,113.550572


In [17]:
out_filename = "test_lb_dist_matrix.csv"
out_path = os.path.join(ES_TEST_RESOURCE_PATH, out_filename)
dist_df.to_csv(out_path)

In [18]:
max_dist_array = 100*np.ones(shape=(len(lb_warn), len(lb_gsr)))
ls_dist_array = np.minimum(dist_array, max_dist_array)
ls_array = ls_vfunc(ls_dist_array)
ls_df = pd.DataFrame(ls_array,
                       index = [w["Warning_ID"] for w in lb_warn],
                       columns = [e["Event_ID"] for e in lb_gsr])

ls_df

,MN0,MN1,MN2,MN3
BR_0,0.759420,0.401820,0.401820,0.611980
BR_1,0.505907,1.000000,1.000000,0.364985
BR_2,0.594347,0.863539,0.863539,0.494531
BR_3,0.550001,0.229456,0.229456,0.656325
BR_4,0.000000,0.000000,0.000000,0.284288
BR_5,0.846592,0.440133,0.440133,0.543234
BR_6,0.777141,0.381911,0.381911,0.568578
BR_7,0.759420,0.401820,0.401820,0.611980
BR_8,0.820949,0.364320,0.364320,0.479976
BR_9,0.000000,0.000000,0.000000,0.000000


In [19]:
out_filename = "test_ls_matrix.csv"
out_path = os.path.join(ES_TEST_RESOURCE_PATH, out_filename)
ls_df.to_csv(out_path)

In [20]:
date_diff_array = [date_diff_to_warn(w, lb_gsr) for w in lb_warn]
date_diff_array = np.array(date_diff_array).reshape(len(lb_warn), len(lb_gsr))

date_diff_df = pd.DataFrame(date_diff_array,
                       index = [w["Warning_ID"] for w in lb_warn],
                       columns = [e["Event_ID"] for e in lb_gsr])

date_diff_df

,MN0,MN1,MN2,MN3
BR_0,1,-7,-7,-3
BR_1,2,-6,-6,-2
BR_2,10,2,2,6
BR_3,7,-1,-1,3
BR_4,6,-2,-2,2
BR_5,4,-4,-4,0
BR_6,-2,-10,-10,-6
BR_7,9,1,1,5
BR_8,0,-8,-8,-4
BR_9,1,-7,-7,-3


In [21]:
out_filename = "test_lb_date_diff_matrix.csv"
out_path = os.path.join(ES_TEST_RESOURCE_PATH, out_filename)
date_diff_df.to_csv(out_path)

In [22]:
date_diff_array = np.abs(date_diff_array)
max_dd_array = 4*np.ones(shape=(len(lb_warn), len(lb_gsr)))
min_dd_array = np.zeros(shape=(len(lb_warn), len(lb_gsr)))
ds_dd_array = np.minimum(date_diff_array, max_dd_array)

ds_dd_array = np.maximum(ds_dd_array, min_dd_array)

ds_array = ds_vfunc(ds_dd_array)
ds_df = pd.DataFrame(ds_array,
                       index = [w["Warning_ID"] for w in lb_warn],
                       columns = [e["Event_ID"] for e in lb_gsr])

ds_df

,MN0,MN1,MN2,MN3
BR_0,0.75,0.00,0.00,0.25
BR_1,0.50,0.00,0.00,0.50
BR_2,0.00,0.50,0.50,0.00
BR_3,0.00,0.75,0.75,0.25
BR_4,0.00,0.50,0.50,0.50
BR_5,0.00,0.00,0.00,1.00
BR_6,0.50,0.00,0.00,0.00
BR_7,0.00,0.75,0.75,0.00
BR_8,1.00,0.00,0.00,0.00
BR_9,0.75,0.00,0.00,0.25


In [23]:
out_filename = "test_ds_matrix.csv"
out_path = os.path.join(ES_TEST_RESOURCE_PATH, out_filename)
ds_df.to_csv(out_path)

In [24]:
es_array = [es_match_to_warn(w, lb_gsr) for w in lb_warn]
es_array = np.array(es_array).reshape(len(lb_warn), len(lb_gsr))

es_df = pd.DataFrame(es_array,
                       index = [w["Warning_ID"] for w in lb_warn],
                       columns = [e["Event_ID"] for e in lb_gsr])

es_df

,MN0,MN1,MN2,MN3
BR_0,0,1,0,1
BR_1,1,0,1,0
BR_2,1,0,1,0
BR_3,1,0,1,0
BR_4,1,0,1,0
BR_5,1,0,1,0
BR_6,1,0,1,0
BR_7,0,1,0,1
BR_8,1,0,1,0
BR_9,1,0,1,0


In [25]:
out_filename = "test_es_match_matrix.csv"
out_path = os.path.join(ES_TEST_RESOURCE_PATH, out_filename)
es_df.to_csv(out_path)

In [26]:
acs_array = [actor_match_to_warn(w, lb_gsr) for w in lb_warn]
acs_array = np.array(acs_array).reshape(len(lb_warn), len(lb_gsr))
acs_df = pd.DataFrame(acs_array,
                       index = [w["Warning_ID"] for w in lb_warn],
                       columns = [e["Event_ID"] for e in lb_gsr])

acs_df

,MN0,MN1,MN2,MN3
BR_0,0,0,0,0
BR_1,0,0,0,0
BR_2,1,1,1,1
BR_3,0,0,0,0
BR_4,0,0,0,0
BR_5,1,1,1,1
BR_6,1,1,1,1
BR_7,1,1,1,1
BR_8,1,1,1,1
BR_9,1,1,1,1


In [27]:
out_filename = "test_actor_match_matrix.csv"
out_path = os.path.join(ES_TEST_RESOURCE_PATH, out_filename)
acs_df.to_csv(out_path)

In [28]:
is_approx_list = [eval(e["Approximate_Location"]) for e in lb_gsr]
is_approx_array = np.array(is_approx_list*len(lb_warn)).reshape(len(lb_warn), len(lb_gsr))
is_approx_df = pd.DataFrame(is_approx_array,
                       index = [w["Warning_ID"] for w in lb_warn],
                       columns = [e["Event_ID"] for e in lb_gsr])

is_approx_df

,MN0,MN1,MN2,MN3
BR_0,False,False,False,False
BR_1,False,False,False,False
BR_2,False,False,False,False
BR_3,False,False,False,False
BR_4,False,False,False,False
BR_5,False,False,False,False
BR_6,False,False,False,False
BR_7,False,False,False,False
BR_8,False,False,False,False
BR_9,False,False,False,False


In [29]:
out_filename = "test_approx_location_matrix.csv"
out_path = os.path.join(ES_TEST_RESOURCE_PATH, out_filename)
is_approx_df.to_csv(out_path)

In [30]:
qs_mat = ls_array + ds_array + es_array + acs_array
qs_mat

array([[ 1.5094204 ,  1.40182019,  0.40182019,  1.86197972],
       [ 2.0059068 ,  1.        ,  2.        ,  0.86498534],
       [ 2.59434716,  2.36353906,  3.36353906,  1.49453079],
       [ 1.55000112,  0.97945638,  1.97945638,  0.90632542],
       [ 1.        ,  0.5       ,  1.5       ,  0.7842882 ],
       [ 2.84659243,  1.44013288,  2.44013288,  2.54323386],
       [ 3.27714139,  1.3819106 ,  2.3819106 ,  1.56857783],
       [ 1.7594204 ,  3.15182019,  2.15182019,  2.61197972],
       [ 3.82094905,  1.36431994,  2.36431994,  1.47997605],
       [ 2.75      ,  1.        ,  2.        ,  1.25      ],
       [ 1.54959528,  1.1897738 ,  2.1897738 ,  1.17178299],
       [ 2.15058152,  2.41643536,  1.41643536,  3.18630744],
       [ 1.59318981,  2.42126871,  1.42126871,  3.82358553],
       [ 2.5059068 ,  2.5       ,  3.5       ,  1.86498534],
       [ 3.7594204 ,  1.40182019,  2.40182019,  1.61197972],
       [ 1.97565804,  2.79486171,  1.79486171,  2.53766211],
       [ 1.76197878,  3.

In [31]:
qs_mat[ls_array == 0] = 0
qs_mat[ds_array == 0] = 0
qs_df = pd.DataFrame(qs_mat,
                       index = [w["Warning_ID"] for w in lb_warn],
                       columns = [e["Event_ID"] for e in lb_gsr])

qs_df

,MN0,MN1,MN2,MN3
BR_0,1.509420,0.000000,0.000000,1.861980
BR_1,2.005907,0.000000,0.000000,0.864985
BR_2,0.000000,2.363539,3.363539,0.000000
BR_3,0.000000,0.979456,1.979456,0.906325
BR_4,0.000000,0.000000,0.000000,0.784288
BR_5,0.000000,0.000000,0.000000,2.543234
BR_6,3.277141,0.000000,0.000000,0.000000
BR_7,0.000000,3.151820,2.151820,0.000000
BR_8,3.820949,0.000000,0.000000,0.000000
BR_9,0.000000,0.000000,0.000000,0.000000


In [32]:
out_filename = "test_qs_mat.csv"
out_path = os.path.join(ES_TEST_RESOURCE_PATH, out_filename)
qs_df.to_csv(out_path)

In [36]:
np.mean((3.8209, 3.3260, 3.5, 3.8236))

3.6176250000000003

In [34]:
4./17

0.23529411764705882

In [35]:
2*(4./17)*1/(1+4./17)

0.38095238095238093